In [1]:
from filterpy.kalman import KalmanFilter
from sklearn.utils.linear_assignment_ import linear_assignment
import numpy as np
from sort_utils import iou, convert_bbox_to_z, convert_x_to_bbox

In [3]:
def init_kalman(bbox):
    kf = KalmanFilter(dim_x=7, dim_z=4)
    kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])
    kf.R[2:,2:] *= 10.
    kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    kf.P *= 10.
    kf.Q[-1,-1] *= 0.01
    kf.Q[4:,4:] *= 0.01
    kf.x[:4] = convert_bbox_to_z(bbox)
    return kf

def update(k, bbox):
    k.update(convert_bbox_to_z(bbox))
    if((k.x[6]+k.x[2])<=0):
        k.x[6] *= 0.0
    k.predict()
    pre_bbox = convert_x_to_bbox(k.x)
    return k, pre_bbox

In [31]:
a = 0.03
b = 0.04
print(np.square(a - b))

0.00010000000000000005


In [41]:
detections = [1,2,3]
trackers = [1.2,3.3]
iou_matrix = np.zeros((len(detections),len(trackers)))
for d, det in enumerate(detections):
    for t, trk in enumerate(trackers):
        iou_matrix[d,t] = np.abs(det - trk)
print(iou_matrix)
matched_indices = linear_assignment(iou_matrix)
print(matched_indices)
# print(matched_indices[:,0])
# print(matched_indices[:,1])
for i in matched_indices:
    print(i)
matched_indices.update()

[[0.2 2.3]
 [0.8 1.3]
 [1.8 0.3]]
[[0 0]
 [2 1]]
[0 0]
[2 1]


AttributeError: 'numpy.ndarray' object has no attribute 'update'

In [ ]:
eg_bbox_0 = np.array([1,2,3,4])
eg_bbox_1 = np.array([2,3,4,5])
eg_bbox_2 = np.array([3,4,5,6])
eg_bbox_3 = np.array([4,5,6,7])
eg_bbox_4 = np.array([5,6,7,8])
k = init_kalman([1,2,3,4])
print(convert_x_to_bbox(k.x))

In [ ]:
k1 = update(k,np.array([2,3,4,5]))

In [ ]:
print(k)

In [ ]:
k = update(k,np.array([3,4,5,6]))

In [ ]:
k = update(k,np.array([4,5,6,7]))

In [ ]:
k = update(k,np.array([5,6,7,8]))

In [ ]:
k = update(k,np.array([6,7,8,9]))

In [ ]:
k = update(k,np.array([7,8,9,10]))

In [ ]:
k = update(k,np.array([8,9,10,11]))

In [ ]:
k = update(k,np.array([9,10,11,12]))